In [ ]:
#!pip install linearmodels --user

In [2]:
import random
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize_scalar
import pandas as pd
#import statsmodels.api as sm
from linearmodels import PanelOLS
import time

In [4]:
#Paramètre du modèle + génération des espilon
random.seed(2040)
n=100
T=10
phizero=0.4
a=np.array([np.random.normal(0,1) for i in range(n)])
y0=np.array([np.random.normal(a[i]/(1-phizero),1/np.sqrt(1-phizero**2)) for i in range(n)])
epsilon=[np.random.normal(0,1) for i in range(n*T)]

On génère les données dans une matrice y de taille $N \times T$ avec en ligne les i et en colonne les T selon la formule :
$y_i,t = y_{i,t-1}+a_i+\epsilon_{i,t}$

In [5]:
y=np.zeros((T,n))
y[0]=y0
y=y.T
for i in range(n):
    for j in range(1,T):
        y[i][j]=a[i]+phizero*y[i][j-1]+epsilon[i*T+j]

yit=y.reshape(n*T)

In [6]:
#Generation des first difference :
delta_y=np.zeros((n,T-1))
for i in range(n):
    for j in range(T-1):
        delta_y[i,j]=y[i,j+1]-y[i,j]

On cheche à coder un GMM baser sur ces moments là : $\forall t>1, \forall s>2, E(y_{it-s}\Delta u_{it})=0$. 

Leur contrepartie empirique est : $$ \frac{1}{N}\sum_{i=1}^N y_{it-s}(\Delta u_{it}) = \frac{1}{N}\sum_{i=1}^N y_{it-s}(\Delta y_{it} - \phi_0 \Delta y_{it-1}) $$

Comme on instrumente directement les différences premières, on ne considère plus l'observation à t=0

$Z_{it}$ contient la liste des instruments de l'individu i au temps t. 

Par exemple : pour t=1, pas d'instrument (car s>2) => on ajoute un 0. Pour t=2, le seul instrument disponible est $y_{i0}$...

Donc sur la ligne t de Z on aura : $(0,y_{i0},(y_{i0},y_{i1}),(y_{i0},y_{i1},y_{i2}),...)$ et ce pour les n individus



In [7]:
z=[]
y
#On parcours tous les individus
for i in range(n):
    #On ajoute une ligne avec un zéros qui symbolise t=1
    z.append([[0]])
    
    #Pour tous les temps T>2, on ajoute les instruments jusqu'à yi0
    for t in range(2,T):
        instrument=[y[i,t-s] for s in range(2,t+1) ]
        z[i].append(instrument)


On a donc $h(\rho,\Delta y_{it},z_{it})$ tel que : $E(h(\rho,\Delta y_{it},z_{it}))=0$.

En l'occurence :  $ h(\rho,\Delta y_{it},z_{it}) = z_{it}(\Delta y_{it}-\rho \Delta y_{it-s})$.

Ici la fonction h stock l'ensemble des moments générés pour l'observation $y_{i,t}$ (un par instrument)




In [8]:
def h(rho,i,t):
    instr=z[i][t]
    return [inst*(delta_y[i,t]-rho*delta_y[i,t-1]) for inst in instr]


Puis on a : 
$h(\rho,Y_i,Z_i,X_i) = (h(\rho,\Delta y_{11},z_{1T}),h(\rho,\Delta y_{21},z_{2T})...)$

Finalement on minimise le critère : 
    $$Q(\theta)=[\sum_{i=1}^Nh_i(\theta,Y_i)]'W_T[\sum_{i=1}^Nh(\theta,Y_i)]$$

In [9]:
#On agrège tous les moments d'un individu donné dans un vecteur 
def tab_h(i,rho):
    tab_h=[]
    for t in range(1,T-1):
        tab_h+=h(rho,i,t)
    return np.array(tab_h)

#On définit le critère ci dessus.
def critere(rho):
    M=1/(n-1)*sum([tab_h(i,rho) for i in range(n-1)])
    return M@M.T


In [10]:
minimize_scalar(critere)

     fun: 0.18450038965338192
    nfev: 11
     nit: 7
 success: True
       x: 0.10770350510053243

Après trop de temps à me perdre dans l'article de Arellano Bover 95 (pour qqch de vraiment pas ouf), j'ai enfin compris que Arrelo Bover ajoute comme nouveaux moments à rajouter dans le GMM. J'ai trouvé cette version clair de ça dans Blundell bond 98 :
$$ E(u_{i,t},\Delta y_{i,t-1})=E((y_{i,t}-\phi_0 y_{it-1})(y_{i,t-1}-y_{it-2})=0, \text{for } t=2,3,...,T $$

Genre les résidu de l'équation au temps t sont orthogonaux de leur "condition intiale" au temps t-1.

On va coder ça. Comme il y qu'un seul momen par temps c'est plus facile, on les mets direct dans un vecteur pour chaque individu.

In [11]:
def moment_l (i,rho):
    moment_l=[]
    for t in range(3,T):
        moment_l+=[(y[i,t]-rho*y[i,t-1])*(y[i,t-1]-y[i,t-2])] 
    return moment_l


def GMM_Ar_Bond (rho) :
    moments=[]
    for i in range(n):
        moments.append(np.array(moment_l(i,rho)+list(tab_h(i,rho))))
    M=1/(n-1)*sum([moments[i] for i in range(n-1)])
    return M@M.T




In [12]:
minimize_scalar(GMM_Ar_Bond)

     fun: 0.20533724450183236
    nfev: 9
     nit: 5
 success: True
       x: 0.174765839635091

Bon pour l'instant je sais pas pourquoi mais les résultats sont absolument pas ouf XD. Je pense que si on remettait tout ca en forme matricielle et qu'on fait juste la formule ca fonctionnerait mieux :(

- Approche Han et Phillips

In [13]:
def HP_estim_direct(y,n,T):
    #Implémentation directe
    numerateur=0
    denominateur=0
    for i in range(n) :
        for t in range(T):
            delta_1=y[i,t-1]-y[i,t-2]
            delta=y[i,t]-y[i,t-1]
            numerateur+=delta_1*(2*delta+delta_1)
            denominateur+=(delta_1)**2

    estim=numerateur/denominateur
   # print("Estimation directe: ",estim)
    return estim

def HP_estim_algo(y,n,T):
    #implémentation par algo de maximisation
    def moment_HP(i,rho):
        moment_l=[]
        for t in range(3,T):
            delta_1=y[i,t-1]-y[i,t-2]
            delta=y[i,t]-y[i,t-1]
            moment_l+=[delta_1*(2*delta+delta_1)-rho*delta_1] 
        return moment_l

    def GMM_HP(rho):
        moments=[]
        for i in range(n):
            moments.append(np.array(moment_HP(i,rho)))
        M=1/(n-1)*sum([moments[i] for i in range(n-1)])
        return M@M.T

    estim2=minimize_scalar(GMM_HP, tol=0.01)
    print("Estimation par algo: ",estim2["x"])  
    
    return(estim2)

HP_estim_direct(y,n,T)
HP_estim_algo(y,n,T)


Estimation par algo:  6.071266693300915


     fun: 0.058041601014900406
    nfev: 10
     nit: 4
 success: True
       x: 6.071266693300915

MLE avec correction de biais de Han et Kuersteiner : la formule explicite est donnée directe dans l'article : je recopie.

In [14]:
def HK_estim(y,n,T):
    within_ = [1/T*sum([y[i][t-1] for t in range(1,T)]) for i in range(n)]
    within = [1/T*sum([y[i][t] for t in range(1,T)]) for i in range(n)]

    #Denominateur (le même pour OLS et HK, noté Upsilon):
    Upsilon=0
    for i in range(n):
        for t in range(1,T):
            Upsilon+=(y[i,t-1]-within_[i])**2

    Upsilon*=1/(n*T)

    #Numérateur (le même pour OLS et la première partie de HK)
    HK_num_1=0
    for i in range(n):
        for t in range(1,T):
            HK_num_1+=(y[i,t-1]-within_[i])*(y[i,t]-within_[i])
    HK_num_1*=1/(n*T)

    phi_OLS=HK_num_1/Upsilon

    omega=(1-phi_OLS**2)*Upsilon

    HK_num_2 = (1/T)*(1-phi_OLS)*omega

    HK_estim=(HK_num_1+HK_num_2)/Upsilon
    
    return HK_estim

print(HK_estim(y,n,T))


0.3072813717645269


Mnt il faut coder l'inférence indirecte :

1) On crée une fonction qui génère des données avec un phi donné

2) On crée une fonction qui calcul OLS avec des données donnée

3) On utilise la super fonction trop cool qui minimise des fonctions

In [6]:
#Exactement la même chose qu'au début
random.seed(2040)
def gener_donne(phi,n,T):
    phizero=float(phi)
    a=np.array([np.random.normal(0,1) for i in range(n)])
    y0=np.array([np.random.normal(a[i]/(1-phizero),1/np.sqrt(1-phizero**2)) for i in range(n)])
    epsilon=[np.random.normal(0,1) for i in range(n*T)]
    y=np.zeros((T,n))
    y[0]=y0
    y=y.T
    for i in range(n):
        for j in range(1,T):
            y[i][j]=a[i]+phizero*y[i][j-1]+epsilon[i*T+j]
    
    return y



In [28]:
def gener_donne_partielle(phi,n,T,a,y0_,espsilon):
    phizero=float(phi)
    y=np.zeros((T,n))
    y0=(y0_+a/(1-phizero))*np.sqrt(1/np.sqrt(1-phizero**2))
    y[0]=y0
    y=y.T
    for i in range(n):
        for j in range(1,T):
            y[i][j]=a[i]+phizero*y[i][j-1]+epsilon[i*T+j]
    
    return y
    

In [12]:
#Formule du papier pour l'estimateur
def ML_papier(y,n,T):
    mat_y_=y[:,:T-1]
    mat_y=y[:,1:]
    
    y=mat_y.reshape(n*(T-1),1)
    y_=mat_y_.reshape(n*(T-1),1)
        
    phi_ML_aux=np.linalg.inv((y_.T)@y_)@((y_.T)@y)
    
    return phi_ML_aux

#Calcul de la binding function simulée pour un phi donnée
def b_NT(phi,H,n,T,a,y0_,epsilon):
    b_simul=[ML_papier(gener_donne_partielle(phi,n,T,a[H],y0_[H],epsilon[H]),n,T) for H in range(H)]
    return np.mean(b_simul)

In [19]:
#Minimisation de l'écart entre la binding fuction simulée et la "vraie valeur"
# de la binding function

def estim_indirect(H,y,n,T):
    x=ML_papier(y,n,T)
    
    a=[np.array([np.random.normal(0,1) for i in range(n)]) for h in range(H)]
    y0=[np.array([np.random.normal(0,1) for i in range(n)]) for h in range(H)]
    epsilon=[[np.random.normal(0,1) for i in range(n*T)] for h in range(H)]
    
    def ecartL2(phi):
        return abs(x-b_NT(phi,H,n,T,a,y0,epsilon))**2
    
    return minimize_scalar(ecartL2,method="bounded", bounds=(0.01,0.99)).x[0][0]


estim_indirect(100,y,n,T)

0.3928124420118898

Même chose avec une autre fonction pour calculer binding function (theta)

In [14]:
#autre fonction pour calculer la stat (LSDV regression, voir Hsiao, Pesaran 1999)
def theta(y,T):
    
    mat_y_=y[:,:T-1] #from 0 to T-1 
    mat_y=y[:,1:] #from 1 to T
    
    mean_y_ = mat_y_.mean(axis=1) #mean from 0 to t-1
    mean_y = mat_y.mean(axis=1) #mean from 1 to T
    
    mat_y_centered = mat_y_ - mean_y_[:,None]
    mat_ycentered = mat_y - mean_y[:,None]
    
    #element product
    mat_numerator = mat_y_centered*mat_ycentered
    mat_denominator = mat_y_centered*mat_y_centered
    
    #estimator (theta)
    theta = np.sum(mat_numerator)/np.sum(mat_denominator)
    return(theta)

In [22]:
def b_NT_bis(phi,H,n,T,a,y0_,epsilon):
    #b_simul=[ML_papier(gener_donne(phi,n,T),n,T) for H in range(H)]
    b_simul=[theta(gener_donne(phi,n,T),T) for H in range(H)]
    return np.mean(b_simul)

#Minimisation de l'écart entre la binding fuction simulée et la "vraie valeur"
# de la binding function

def estim_indirect_bis(H,y,n,T):
    #x=ML_papier(y,n,T)
    x=theta(y,T)
    
    a=[np.array([np.random.normal(0,1) for i in range(n)]) for h in range(H)]
    y0=[np.array([np.random.normal(0,1) for i in range(n)]) for h in range(H)]
    epsilon=[[np.random.normal(0,1) for i in range(n*T)] for h in range(H)]
    
    def ecartL2(phi):
        return (x-b_NT_bis(phi,H,n,T,a,y0,epsilon))**2
    
    opt = minimize_scalar(ecartL2,method="bounded", bounds=(0.01,0.99))
    return(opt.x)
    #return minimize_scalar(ecartL2,method="bounded", bounds=(0.01,0.99)).x[0][0]

In [23]:
start_time = time.time()
print(estim_indirect(10,y,n,T))
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
print(estim_indirect_bis(10,y,n,T))
print("--- %s seconds ---" % (time.time() - start_time))

0.4045913092324588
--- 1.0850567817687988 seconds ---
0.4185955421380356
--- 1.2572786808013916 seconds ---


Calcul de la RMSE

In [18]:
def RMSE(phi_estim_vec,phi):
    return np.sqrt(np.mean((np.array(phi_estim_vec)-np.array([phi]*len(phi_estim_vec)))**2))


RMSE([1,0.6,0],0.5)

0.41231056256176607

In [21]:
T_liste=[5]*8+[10]*8+[20]*8
N_liste=(([100]*4+[200]*4)*4)
phi_liste=[0,0.3,0.6,0.9]*6

def estim_ind(H):
    def estim_ind (y,n,T):
        return estim_indirect(H,y,n,T)
    
    return estim_ind

#Fonction qui prend nb de replic, phi, n, T, l'estimateur

def replic_estim(n_replic,phi,n,T,fonction, verbose=False) :
    estim=[]
    for i in range(n_replic) :
        y=gener_donne(phi,n,T)
        estim+=[fonction(y,n,T)] 
        if(verbose):
            print(i)
    biais=np.mean(np.array(estim))-phi
    RMSE_mod=RMSE(estim,phi)
        
    return biais,RMSE_mod

print(replic_estim(n_replic=1000,phi=0.5,n=100,T=10,fonction=HK_estim))
print(replic_estim(n_replic=100,phi=0.5,n=100,T=10,fonction=HP_estim_direct))
print(replic_estim(n_replic=100,phi=0.5,n=100,T=10,fonction=estim_ind(5), verbose=True))


KeyboardInterrupt: 